# 概要

一、赛题背景

预测性维护是工业互联网应用“皇冠上的明珠”，实现预测性维护的关键是对设备系统或核心部件的寿命进行有效预测。对工程机械设备的核心耗损性部件的剩余寿命进行预测，可以据此对于相关部件的进行提前维护或者更换，从而减少整个设备非计划停机时间，避免因计划外停机而带来的经济损失，比如导致整个生产现场其他配套设备等待故障设备部件的修复。

二、赛事任务

本赛题由中科云谷科技有限公司提供某类工程机械设备的核心耗损性部件的工作数据，包括部件工作时长、转速、温度、电压、电流等多类工况数据。希望参赛者利用大数据分析、机器学习、深度学习等方法，提取合适的特征、建立合适的寿命预测模型，预测核心耗损性部件的剩余寿命。

三、开放数据

针对某类工程机械设备的核心耗损性部件，数据集包含训练集和测试集两个部分。

训练集中，每个文件对应一个该类部件的全寿命物联网采样数据，即从安装后一直到更换之间的对应数据，形式为多维时间序列。字段“部件工作时长”的最大值（通常为最后一行记录）即为该部件实例的实际寿命。（参见样例数据）

测试集中，每个文件对应一个该类部件一段时间内的物联网采样数据，需要基于该段数据，预测该部件此后的剩余寿命。

特征数据字段包括：部件工作时长, 累积量参数1，累积量参数2，转速信号1, 转速信号2, 压力信号1, 压力信号2, 温度信号, 流量信号, 电流信号, 开关1信号, 开关2信号, 告警信号1, 设备类型。其中：

数值型字段包括：部件工作时长, 累积量参数1，累积量参数2，转速信号1, 转速信号2, 压力信号1, 压力信号2, 温度信号, 流量信号, 电流信号。

开关量字段（0或1）：开关1信号, 开关2信号, 告警信号1

字符串型字段：设备类型。

除了开关量以外，上述设备类型、工况数据的具体值都经过了一定的脱敏处理，但已考虑尽量不影响数据蕴含的关系。

赛题的算法预测精度的衡量标准公式如下：

![工程机械寿命预测公式](工程机械寿命预测公式.png)

其中，ri表示第i个样本的真实剩余寿命，r ̂_i表示第i个样本的预测剩余寿命。

# 特征工程

In [1]:
import os
import numpy as np
import pandas as pd
from itertools import groupby
from scipy.stats import pearsonr

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 获取文件名列表

In [3]:
train_path = './train'
test_path = './test1'

In [4]:
def get_filelist(dir_path, flielist):
    if os.path.isfile(dir_path):
        flielist.append(dir_path)
    elif os.path.isdir(dir_path):
        for s in os.listdir(dir_path):
            new_dir = os.path.join(dir_path, s)
            get_filelist(new_dir, flielist)
            
    return flielist

In [5]:
train_list = get_filelist(train_path, [])
test_list = get_filelist(test_path, [])

## 单个文件处理

**连续负值行填充为前后正常值的均值**

In [6]:
def continuous_negative_pro(df, index, feature_name):
    for key, g_value in groupby(enumerate(index), lambda x: x[1]-x[0]):
        each_g_lst = [element for i, element in g_value]
        if len(each_g_lst) > 1:
            ucl = min(each_g_lst) 
            dcl = max(each_g_lst)
        else:
            ucl = each_g_lst[0]
            dcl = each_g_lst[0]
        try:
            df.iloc[ucl:dcl+1][feature_name] = (df.iloc[ucl-1][feature_name] + df.iloc[dcl+1][feature_name]) / 2
        except Exception:
            df.iloc[ucl:dcl+1][feature_name] = df[feature_name].mean()
        
    return df

In [7]:
def preprocess_single_file(path):
    # 部件工作时长负值处理
    # 删除部件工作时长为负的行
    drop_negative = pd.read_csv(path)
    drop_negative.drop(index=drop_negative['部件工作时长'][drop_negative['部件工作时长']<0].index, inplace=True)
    
    # 按部件工作时长升序重新生成DateFrame
    drop_negative.sort_values(by="部件工作时长", ascending=True, inplace=True, kind='mergesort')
    cum_param1_fill = drop_negative.reset_index(drop=True)
    
    # 累积量参数1负值处理
    # 连续负值行填充为前后正常值的均值
    cum_param1_fill_index = cum_param1_fill['累积量参数1'][cum_param1_fill['累积量参数1']<0].index.sort_values()
    if len(cum_param1_fill_index) > 0:
        cum_param2_fill = continuous_negative_pro(cum_param1_fill, cum_param1_fill_index, '累积量参数1')
    else:
        cum_param2_fill = cum_param1_fill
        
    # 累积量参数2负值处理
    cum_param2_fill_index = cum_param2_fill['累积量参数2'][cum_param2_fill['累积量参数2']<0].index.sort_values()
    if len(cum_param2_fill_index) > 0:
        temperature_fill = continuous_negative_pro(cum_param2_fill, cum_param2_fill_index, '累积量参数2')
    else:
        temperature_fill = cum_param2_fill
        
    # 温度信号负值处理
    temperature_fill_index = temperature_fill['温度信号'][temperature_fill['温度信号']<0].index.sort_values()
    if len(temperature_fill_index) > 0:
        new_raw_data = continuous_negative_pro(temperature_fill, temperature_fill_index, '温度信号')
    else:
        new_raw_data = temperature_fill

    return new_raw_data

## 单项特征处理

In [8]:
# 根据name选择处理特征
def single_feature_project(data, df, name, k):

    # 开关与告警信号取其在总数据中的占比
    if name == '开关1信号' or name == '开关2信号' or name == '告警信号1':
        df[name + '时间占比'] = data.sum() / len(data)

    # 温度信号取其均值、标准差与极差为特征
    elif name == '温度信号':
        df[name + '均值'] = data.mean()
        df[name + '标准差'] = data.std()
        df[name + '极差'] = data.ptp()

    # 累积量参数取最大值、k个周期差分的均值与标准差作为特征
    elif name == '累积量参数1' or name == '累积量参数2':
        df[name] = data.max()
        # 将数据移动k之后与原数据进行比较得出差异数据
        data = data.diff(periods=k)
        data = data.dropna()
        df[name + str(k) + '阶差分均值'] = data.mean()
        df[name + str(k) + '阶差分标准差'] = data.std()

    # 电流信号主要分布在三段区间中，分别取其均值与标准差，加权后作为特征
    elif name == '电流信号':
        length = len(data)
        low_current = list(num for num in data if 0 <= num < 20)
        mid_current = list(num for num in data if 500 <= num < 750)
        high_current = list(num for num in data if 800 <= num < 1800)
        low_percentage = np.sum(low_current) / length
        mid_percentage = np.sum(mid_current) / length
        high_percentage = np.sum(high_current) / length
        df[name + '低电流段均值'] = np.mean(low_current) * low_percentage
        df[name + '中电流段均值'] = np.mean(mid_current) * mid_percentage
        df[name + '高电流段均值'] = np.mean(high_current) * high_percentage
        df[name + '低电流段标准差'] = np.std(low_current) * low_percentage
        df[name + '中电流段标准差'] = np.std(mid_current) * mid_percentage
        df[name + '高电流段标准差'] = np.std(high_current) * high_percentage

    # 流量信号主要分布在三段区间中，分别取其均值与标准差，加权后作为特征
    elif name == '流量信号':
        length = len(data)
        low_current = list(num for num in data if 0 <= num < 9)
        mid_current = list(num for num in data if 10 <= num < 120)
        high_current = list(num for num in data if 125 <= num < 145)
        low_percentage = np.sum(low_current) / length
        mid_percentage = np.sum(mid_current) / length
        high_percentage = np.sum(high_current) / length
        df[name + '低流量段均值'] = np.mean(low_current) * low_percentage
        df[name + '中流量段均值'] = np.mean(mid_current) * mid_percentage
        df[name + '高流量段均值'] = np.mean(high_current) * high_percentage
        df[name + '低流量段标准差'] = np.std(low_current) * low_percentage
        df[name + '中流量段标准差'] = np.std(mid_current) * mid_percentage
        df[name + '高流量段标准差'] = np.std(high_current) * high_percentage

    # 压力信号1主要分布在两段区间上，剩余值较小，取其标准差加权后作为特征
    elif name == '压力信号1':
        length = len(data)
        low_pressure = list(num for num in data if 65 <= num <= 75)
        high_pressure = list(num for num in data if 180 <= num <= 400)
        low_percentage = np.sum(low_pressure) / length
        high_percentage = np.sum(high_pressure) / length
        df[name + '信号1低压力段标准差'] = np.std(low_pressure) * low_percentage
        df[name + '信号1高压力段标准差'] = np.std(high_pressure) * high_percentage

    # 压力信号2处理同上
    elif name == '压力信号2':
        length = len(data)
        low_pressure = list(num for num in data if 0 <= num <= 50)
        high_pressure = list(num for num in data if 200 <= num)
        low_percentage = np.sum(low_pressure) / length
        high_percentage = np.sum(high_pressure) / length
        df[name + '信号2低压力段标准差'] = np.std(low_pressure) * low_percentage
        df[name + '信号2高压力段标准差'] = np.std(high_pressure) * high_percentage

    # 转速信号1主要分布在两段区间上，取其均值与标准差加权后作为特征
    elif name == '转速信号1':
        length = len(data)
        low_pressure = list(num for num in data if 0 <= num <= 100)
        high_pressure = list(num for num in data if 3000 <= num)
        low_percentage = np.sum(low_pressure) / length
        high_percentage = np.sum(high_pressure) / length
        df[name + '信号1低转速段均值'] = np.mean(low_pressure) * low_percentage
        df[name + '信号1高转速段均值'] = np.mean(high_pressure) * high_percentage
        df[name + '信号1低转速段标准差'] = np.std(low_pressure) * low_percentage
        df[name + '信号1高转速段标准差'] = np.std(high_pressure) * high_percentage

    # 转速信号2主要分布在两段区间上，取其均值与标准差加权后作为特征
    elif name == '转速信号2':
        length = len(data)
        low_pressure = list(num for num in data if 0 <= num <= 1000)
        high_pressure = list(num for num in data if 10000 <= num)
        low_percentage = np.sum(low_pressure) / length
        high_percentage = np.sum(high_pressure) / length
        df[name + '信号2极低转速段均值'] = np.mean(low_pressure) * low_percentage
        df[name + '信号2高转速段均值'] = np.mean(high_pressure) * high_percentage
        df[name + '信号2极低转速段标准差'] = np.std(low_pressure) * low_percentage
        df[name + '信号2高转速段标准差'] = np.std(high_pressure) * high_percentage

    return df

## 耦合特征

In [9]:
# 构造转速信号1、转速信号2、压力信号1、压力信号2、温度信号、流量信号、电流信号的耦合特征
def coupled_feature(dataframe, df):
    # 取出列名表
    column_list = dataframe.columns.values.tolist()

    # 循环构造两两乘积特征
    for i in range(3, 10):
        for j in range(i + 1, 10):
            mutiple = dataframe.iloc[:, [i]].values * dataframe.iloc[:, [j]].values
            df[column_list[i] + '与' + column_list[j] + '乘积的均值'] = mutiple.mean()
            df[column_list[i] + '与' + column_list[j] + '乘积的标准差'] = mutiple.std()

    # 循环构造平方项特征
    for i in range(3, 10):
        square = dataframe.iloc[:, [i]].values * dataframe.iloc[:, [i]].values
        df[column_list[i] + '平方项的均值'] = square.mean()
        df[column_list[i] + '平方项的标准差'] = square.std()

    return df

## 单个样本特征提取

In [10]:
def process_single_sample(path, train_percentage, k):
    # 单个文件预处理
    new_raw_data = preprocess_single_file(path)
    # 获取样本部件工作最大时长
    work_life = new_raw_data['部件工作时长'].max()
    # 根据train_percentage分割数据，生成新样本
    new_data = new_raw_data[new_raw_data['部件工作时长'] <= work_life * train_percentage]
    # 创建数据特征字典
    # 设备类型特征相关性低：剔除
    # 开关1信号、开关2信号、告警信号差异较大：剔除
    dict_data = {'train_file_name': os.path.basename(path) + str(train_percentage),
                 'current_life': np.log(new_data['部件工作时长'].max() + 1),
                 'rest_life': np.log(work_life - new_data['部件工作时长'].max() + 1)
                 }
    # 单项特征提取
    for item in ['部件工作时长', '累积量参数1', '累积量参数2', '转速信号1', '转速信号2', '压力信号1', '压力信号2', '温度信号', 
                 '流量信号', '电流信号', '开关1信号', '开关2信号', '告警信号1']:
        dict_data = single_feature_project(new_data[item], dict_data, item, k)
    # 耦合特征提取
    dict_data = coupled_feature(new_data, dict_data)
    # 输出特征df
    single_sample_features = pd.DataFrame(dict_data, index=[0])
    
    return single_sample_features

## 整合创建测试集与训练集

In [11]:
def integrated_process(path_list, test_or_not, k):
    feature_df = pd.DataFrame()

    if test_or_not:
        # 测试集无需对数据进行分割处理
        train_percentage_list = [1]
    else:
        # 训练集目的为预测剩余寿命，故将数据集分割
        train_percentage_list = [0.45, 0.55, 0.63, 0.75, 0.85]

    for path in path_list:
        for train_percentage in train_percentage_list:
            feature_df = feature_df.append(process_single_sample(path, train_percentage, k), ignore_index=True)
            
    columns = feature_df.columns.tolist()
    for col in ['train_file_name', 'rest_life']:
        columns.remove(col)
    columns = ['train_file_name'] + columns + ['rest_life']
    
    if test_or_not:
        feature_df['train_file_name'] = feature_df['train_file_name'].apply(lambda x: x[:-1])
        
    feature_df = feature_df.reindex(columns=columns)

    return feature_df

# 数据建模

## 创建数据集

**时间点保留值**

In [12]:
num_k = 12

### 训练集

In [13]:
train = integrated_process(train_list, False, num_k)

In [14]:
train.head(3)

,train_file_name,current_life,累积量参数1,累积量参数112阶差分均值,累积量参数112阶差分标准差,累积量参数2,累积量参数212阶差分均值,累积量参数212阶差分标准差,转速信号1信号1低转速段均值,转速信号1信号1高转速段均值,...,压力信号1平方项的标准差,压力信号2平方项的均值,压力信号2平方项的标准差,温度信号平方项的均值,温度信号平方项的标准差,流量信号平方项的均值,流量信号平方项的标准差,电流信号平方项的均值,电流信号平方项的标准差,rest_life
0,00fb58ecd675062e4423.csv0.45,8.181301,65560.0,33.940715,158.108275,70109.0,36.295219,158.690456,0.038098,6.198263e+07,...,71708.267142,121337.130736,27555.678773,3696.849270,1335.967940,8052.022287,7000.929978,872179.686018,797500.460414,8.382747
1,00fb58ecd675062e4423.csv0.55,8.380801,80245.0,33.024149,157.125505,85943.0,35.368277,159.312174,0.035908,6.173523e+07,...,80473.078233,119507.691084,26938.164021,3858.818043,1359.968261,7973.518055,6922.163799,860445.942874,791749.318885,8.183677
2,00fb58ecd675062e4423.csv0.63,8.518093,94880.5,32.700511,163.874058,100191.0,34.528764,161.120839,0.039156,6.203548e+07,...,88845.868822,117654.521918,27082.908908,3952.262971,1410.770430,8275.196679,7069.322492,850064.764294,788084.063288,7.986165


### 测试集

In [15]:
test = integrated_process(test_list, True, num_k)

In [16]:
test.head(3)

,train_file_name,current_life,累积量参数1,累积量参数112阶差分均值,累积量参数112阶差分标准差,累积量参数2,累积量参数212阶差分均值,累积量参数212阶差分标准差,转速信号1信号1低转速段均值,转速信号1信号1高转速段均值,...,压力信号1平方项的标准差,压力信号2平方项的均值,压力信号2平方项的标准差,温度信号平方项的均值,温度信号平方项的标准差,流量信号平方项的均值,流量信号平方项的标准差,电流信号平方项的均值,电流信号平方项的标准差,rest_life
0,002ece6be8b41a5613aa.csv,6.144186,6908.0,29.568513,134.437747,7637.5,32.765306,144.481097,0.009774,5.972816e+07,...,42260.900138,121495.907389,35527.019175,2447.471742,1609.752100,10570.471066,6774.181528,9.183055e+05,800452.370241,0.0
1,004a2ad4b735329a3e23.csv,5.961649,5830.0,28.831529,133.456313,5681.0,28.511044,110.416093,0.002348,5.423300e+07,...,40737.218562,130700.673252,32732.798894,2453.532714,662.566167,3157.795409,4675.441443,1.356012e+06,754860.700099,0.0
2,004c675bb05d447aa94b.csv,7.914435,51106.0,4.158292,66.202387,56979.0,4.635855,72.009403,NaN,1.179689e+08,...,107884.064795,151709.150527,37294.542169,3653.465596,1043.198004,11268.020296,6875.748431,1.232041e+06,521014.106011,0.0


### 训练集、测试集合并

In [17]:
train_test = pd.concat([train, test], join='outer', axis=0).reset_index(drop=True)

In [18]:
train_test.tail(3)

,train_file_name,current_life,累积量参数1,累积量参数112阶差分均值,累积量参数112阶差分标准差,累积量参数2,累积量参数212阶差分均值,累积量参数212阶差分标准差,转速信号1信号1低转速段均值,转速信号1信号1高转速段均值,...,压力信号1平方项的标准差,压力信号2平方项的均值,压力信号2平方项的标准差,温度信号平方项的均值,温度信号平方项的标准差,流量信号平方项的均值,流量信号平方项的标准差,电流信号平方项的均值,电流信号平方项的标准差,rest_life
5466,ffb78f9b18e44f78bd49.csv,7.721681,44022.0,37.875423,217.866884,41086.5,35.308850,162.607325,0.020551,5.318975e+07,...,26366.398116,145185.755166,25598.333558,3412.841754,726.129560,5145.434839,4836.751536,8.351913e+05,790801.066760,0.0
5467,ffbefac0e4396bacb898.csv,6.021630,6462.5,31.769800,171.178668,6422.0,32.420801,152.601540,0.022325,6.245496e+07,...,21132.271068,111990.584433,28448.152873,3801.803026,1141.714802,4595.224056,4592.916978,7.967952e+05,732289.335107,0.0
5468,ffe279f5242310018444.csv,6.427297,8272.0,30.765951,115.666400,8164.0,30.791851,103.766986,0.019938,5.861260e+07,...,28493.799239,128793.967107,26425.749230,1878.784103,502.935815,4041.750117,4436.299935,1.467741e+06,636649.009001,0.0


## 数据集整理

### 填充NAN值

In [19]:
train_test.fillna(0, inplace=True)

### 特征列标准化

In [20]:
column_names = train_test.columns.values.tolist()
special_column_names = ['train_file_name'] + ['current_life'] + ['rest_life']
for item in special_column_names:
    column_names.remove(item)

In [21]:
for item in column_names:
    std_temp = train_test[item].std()

    if std_temp <= 1:
        train_test[item] = np.exp(train_test[item])
        std_temp2 = train_test[item].std()

        if std_temp2 < 1:
            del train_test[item]

    elif std_temp > 10:
        train_test[item] = np.log(train_test[item] + 1)

### 皮尔森相关系数

In [22]:
def pearson(data):
    column_num_list = []

    for i in range(2, data.shape[1] - 1):
        a = data.iloc[:, [i]].values.flatten()
        b = data['rest_life'].values.flatten()
        r, p = pearsonr(a, b)
        if r == np.nan:
            column_num_list.append(i)

    data.drop(data.columns[column_num_list], axis=1, inplace=True)
    
    return data

In [23]:
train_test = pearson(train_test)

## LightGBM模型

In [24]:
import lightgbm as lgb
from sklearn.model_selection import KFold

### 模型参数

In [25]:
params_lgb = {'num_leaves': 250,
              'max_depth': 5,
              'learning_rate': 0.01,
              'objective': 'regression',
              'boosting': 'gbdt',
              'verbosity': -1}

In [26]:
fit_params_lgb = {'num_boost_round': 2500,
                  'verbose_eval': 200,
                  'early_stopping_rounds': 200}

### 评价指标

In [27]:
def compute_loss(target, predict):
    temp = np.log(abs(target + 1)) - np.log(abs(predict + 1))
    res = np.sqrt(np.dot(temp, temp) / len(temp))
    return res

### 模型构建

In [28]:
def lgb_cv(train, params, fit_params, feature_names, nfold, seed, test):
    train_pred = pd.DataFrame({
        'true': train['rest_life'],
        'pred': np.zeros(len(train))})
    test_pred = pd.DataFrame({'train_file_name': test['train_file_name'], 'rest_life': np.zeros(len(test))},
                             columns=['train_file_name', 'rest_life'])
    kfolder = KFold(n_splits=nfold, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(kfolder.split(train)):
        print('\nFold_{fold_id} Training ================================\n'.format(fold_id=fold_id))
        lgb_trn = lgb.Dataset(
            data=train.iloc[trn_idx][feature_names],
            label=train.iloc[trn_idx]['rest_life'],
            feature_name=feature_names)
        lgb_val = lgb.Dataset(
            data=train.iloc[val_idx][feature_names],
            label=train.iloc[val_idx]['rest_life'],
            feature_name=feature_names)
        lgb_reg = lgb.train(params=params, train_set=lgb_trn,
                            num_boost_round=fit_params['num_boost_round'], verbose_eval=fit_params['verbose_eval'],
                            early_stopping_rounds=fit_params['early_stopping_rounds'],
                            valid_sets=[lgb_trn, lgb_val])
        val_pred = lgb_reg.predict(
            train.iloc[val_idx][feature_names],
            num_iteration=lgb_reg.best_iteration)

        train_pred.loc[val_idx, 'pred'] = val_pred
        test_pred['rest_life'] += (np.exp(lgb_reg.predict(test[feature_names])) - 1)
    test_pred['rest_life'] = test_pred['rest_life'] / nfold
    score = compute_loss(pd.Series(np.exp(train_pred['true']) - 1).apply(max, args=(0,))
                         , pd.Series(np.exp(train_pred['pred']) - 1).apply(max, args=(0,)))
    print('\nCV LOSS:', score)
    return test_pred

### 模型训练与预测

**nfold交叉验证**

In [29]:
nfold = 5
seed = 1024

In [30]:
feature_name = list(filter(lambda x: x not in ['train_file_name', 'rest_life'], train_test.columns))

In [31]:
df_test_label = lgb_cv(train_test.iloc[:train.shape[0]], params_lgb, fit_params_lgb,
                 feature_name, nfold, seed, train_test.iloc[train.shape[0]:])


Fold_0 Training ================================

Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 0.393209	valid_1's l2: 0.440011
[400]	training's l2: 0.320958	valid_1's l2: 0.415019
[600]	training's l2: 0.297575	valid_1's l2: 0.413335
Early stopping, best iteration is:
[566]	training's l2: 0.301118	valid_1's l2: 0.413161

Fold_1 Training ================================

Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 0.381288	valid_1's l2: 0.584512
[400]	training's l2: 0.316427	valid_1's l2: 0.516063
[600]	training's l2: 0.294566	valid_1's l2: 0.486324
[800]	training's l2: 0.275483	valid_1's l2: 0.465816
[1000]	training's l2: 0.258278	valid_1's l2: 0.44885
[1200]	training's l2: 0.238208	valid_1's l2: 0.431735
[1400]	training's l2: 0.219275	valid_1's l2: 0.41776
[1600]	training's l2: 0.197115	valid_1's l2: 0.403789
[1800]	training's l2: 0.175129	valid_1's l2: 0.391845
[2000]	training's l2: 0.156313	valid_1's l2: 

### 输出文件

In [32]:
df_test_label.to_csv('./test_result/df_test_label_lgb_0808_3.csv', index=False)

In [33]:
df_test_label.describe()

,rest_life
count,889.000000
mean,1052.908024
std,748.845885
min,111.689409
25%,347.457088
50%,964.332323
75%,1653.859723
max,5292.937373
